<a href="https://colab.research.google.com/github/kridtapon/SuperTrix/blob/main/SuperTrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Supertrend Indicator
def calculate_supertrend(df, period=10, multiplier=3):
    """
    Calculate Supertrend Indicator.
    """
    df['ATR'] = df['High'].rolling(period).max() - df['Low'].rolling(period).min()
    df['ATR'] = df['ATR'].ewm(span=period).mean()  # Smoothed ATR

    hl2 = (df['High'] + df['Low']) / 2
    df['UpperBand'] = hl2 + (multiplier * df['ATR'])
    df['LowerBand'] = hl2 - (multiplier * df['ATR'])

    df['Supertrend'] = np.nan
    trend = True  # True = Bullish, False = Bearish

    for i in range(1, len(df)):
        if df['Close'][i] > df['UpperBand'][i-1]:
            trend = True
        elif df['Close'][i] < df['LowerBand'][i-1]:
            trend = False

        df.loc[df.index[i], 'Supertrend'] = df['UpperBand'][i] if trend == False else df['LowerBand'][i]

    df['Supertrend_Up'] = df['Supertrend'].where(df['Supertrend'] < df['Close'])
    df['Supertrend_Down'] = df['Supertrend'].where(df['Supertrend'] > df['Close'])

    return df

# Function to calculate Trix Indicator
def calculate_trix(df, period=15):
    """
    Calculate the Trix (Triple Exponential Average).
    """
    trix = df['Close'].ewm(span=period).mean()
    trix = trix.ewm(span=period).mean()
    trix = trix.ewm(span=period).mean()
    trix = trix.pct_change() * 100  # Convert to percentage change
    return trix

# Define the stock symbol and time period
symbol = 'META'
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Supertrend
df = calculate_supertrend(df, period=10, multiplier=2)

# Calculate Trix
df['Trix'] = calculate_trix(df, period=15)

# Define Entry & Exit conditions based on Supertrend + Trix strategy
df['Entry'] = (df['Close'] > df['Supertrend']) & (df['Trix'] > 0)
df['Exit'] = (df['Close'] < df['Supertrend']) & (df['Trix'] < 0)

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


Streaming output truncated to the last 5000 lines.
<ipython-input-3-05edf7995ceb>:24: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-3-05edf7995ceb>:27: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-3-05edf7995ceb>:22: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

<ipython-input-3-05edf7995ceb>:24: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer k

Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           351826.983613
Total Return [%]                       251.826984
Benchmark Return [%]                   180.172856
Max Gross Exposure [%]                      100.0
Total Fees Paid                        326.122973
Max Drawdown [%]                        44.237612
Max Drawdown Duration                       462.0
Total Trades                                    2
Total Closed Trades                             1
Total Open Trades                               1
Open Trade PnL                      238715.660067
Win Rate [%]                                100.0
Best Trade [%]                          13.124435
Worst Trade [%]                         13.124435
Avg Winning Trade [%]                   13.124435
Avg Losing Trade [%]                          NaN
